In [166]:
import numpy as np
from qiskit_aer import Aer
from qiskit.circuit.library import QFT, PhaseEstimation, UnitaryGate
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister, ClassicalRegister
from scipy.linalg import expm
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import scipy.linalg



class HHL():

    def __init__( self, epsilon: float = 1e-2) -> None:

        super().__init__()

        self._epsilon = epsilon
        # Tolerance for the different parts of the algorithm as per [1]
        self._epsilon_r = epsilon / 3  # conditioned rotation
        self._epsilon_s = epsilon / 3  # state preparation
        self._epsilon_a = epsilon / 6  # hamiltonian simulation
        self.b = None
        self.b_norm=None

        self._scaling = None  # scaling of the solution

        self._sampler = None
        self.num_shots=10000
        # For now the default reciprocal implementation is exact
        self._exact_reciprocal = True
        # Set the default scaling to 1
        self.scaling = 1
        self.observable = None
        self.x = None

    def get_observable(self,qc):
        
        simulator = AerSimulator()
        circ = transpile(qc, simulator)

        # Run and get counts
        result = simulator.run(circ, shots=self.num_shots).result()
        counts = result.get_counts(circ)
        print(counts)
        self.observable = counts


    def construct_circuit(self, matrix: np.ndarray,vector: np.ndarray) -> QuantumCircuit:

        # State preparation circuit - default is qiskit
        nb = int(np.log2(len(vector)))
        vector_circuit = QuantumCircuit(nb)
        self.b_norm = np.linalg.norm(vector)
        vector_circuit.initialize(vector / self.b_norm, list(range(nb)), None)

        # If state preparation is probabilistic the number of qubit flags should increase
        nf = 1

        # Hamiltonian simulation circuit
        matrix = np.array(matrix)  # Ensure the matrix has a compatible type
        #matrix_circuit = NumPyMatrix(matrix, evolution_time=2 * np.pi)

        #---------------------------------------------------------------------------------------------------------------

        # check if the matrix can calculate the condition number and store the upper bound
        kappa = np.linalg.cond(matrix)

        # Update the number of qubits required to represent the eigenvalues
        # The +neg_vals is to register negative eigenvalues because
        # e^{-2 \pi i \lambda} = e^{2 \pi i (1 - \lambda)}
        nl = max(nb + 1, int(np.ceil(np.log2(kappa + 1)))) #+ neg_vals
        

        #--------------------------------------------------------------------------------------------------------------

        # check if the matrix can calculate bounds for the eigenvalues
        lambda_min, lambda_max = min(np.abs(np.linalg.eigvals(matrix))), max(np.abs(np.linalg.eigvals(matrix)))
        matrix = matrix / lambda_max
        U = expm(-1j * matrix * 2 * np.pi)  # Unitary matrix corresponding to e^(-i*A*2π)
        U_circuit = UnitaryGate(U)

        # Constant so that the minimum eigenvalue is represented exactly, since it contributes
        
        # Update the scaling of the solution
        self.scaling = lambda_min

        # Initialise the quantum registers
        qb = QuantumRegister(nb)  # right hand side and solution
        ql = QuantumRegister(nl)  # eigenvalue evaluation qubits
        qf = QuantumRegister(nf)  # flag qubits
        cl = ClassicalRegister(nl)

        qc = QuantumCircuit(qb, ql, qf,cl)

        # State preparation
        qc.append(vector_circuit, qb[:])
        # QPE
        phase_estimation = PhaseEstimation(nl, U_circuit)

        qc.append(phase_estimation, ql[:] + qb[:])

        

        qc.measure(ql,cl) 

        self.get_observable(qc)

        return qc

    def solve(
        self,
        matrix: np.ndarray,
        vector:  np.ndarray,
    ):
        self.b = vector
        solution = HHL()
        solution.state = self.construct_circuit(matrix, vector)

        return solution

In [167]:
# Example usage
A = [[1/4   ,0   ],
     [0   ,1/2  ]]


b = [1,2]

hhl = HHL()
solution = hhl.solve(A, b)
print(solution.state)

{'00': 7970, '01': 2030}
        ┌──────────────┐┌──────┐      
  q250: ┤ circuit-4925 ├┤2     ├──────
        └──────────────┘│      │┌─┐   
q251_0: ────────────────┤0 QPE ├┤M├───
                        │      │└╥┘┌─┐
q251_1: ────────────────┤1     ├─╫─┤M├
                        └──────┘ ║ └╥┘
  q252: ─────────────────────────╫──╫─
                                 ║  ║ 
 c83: 2/═════════════════════════╩══╩═
                                 0  1 
